# YOLO Training & Testing

### Imports

In [1]:
import os
import shutil

### GPUs, HF Cache

In [2]:
%matplotlib inline 

# Model
%env CUDA_VISIBLE_DEVICES=1
#%env PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:32

# HF Cache
os.environ["HF_HOME"] = "../../.cache"
!echo $HF_HOME
!huggingface-cli whoami

env: CUDA_VISIBLE_DEVICES=1
../../.cache
Maats
orgs:  DBD-research-group,Basket-AEye


### Select MVTec with specific Labels

In [ ]:
IMG_SIZE = 500
EPOCHS = 3
DATASET_PATH = "../../huggingface/ai_shelf/artificial_created_dataset/atrifitial_created_dataset.yaml"
NAME = "first_atrificial_dataset_full_run" # For WANDB (YOLO11n-COCO11-on_mvtec_train_with_augmented)

In [ ]:
from ultralytics import YOLO
import wandb
from datetime import datetime

# WANB Init with custom name 
wandb.init(
    project="Yolo-Training",
    entity="maats",
    name=NAME+datetime.now().strftime("%d%b-%H:%M:%S"),
    config={  # alle Hyperparameter sauber abspeichern
        "epochs": EPOCHS,
        "imgsz": IMG_SIZE,
        "model": "yolo11n.pt",
        "dataset": DATASET_PATH,
    },
    sync_tensorboard=True,
)

try:
    # YOLO-Model load
    model = YOLO("yolo11n.pt")

    # Training
    results = model.train(
        data=DATASET_PATH,
        epochs=EPOCHS,
        imgsz=IMG_SIZE,
        project="runs/train",
        name=NAME,
        verbose=True,
        val=True,
        save=True,
        save_period=3,
        mode="wandb"
    )

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # wandb.finish()
    # print("done")
    pass 


wandb: Currently logged in as: torge-schwark (maats). Use `wandb login --relogin` to force relogin
E0000 00:00:1749649504.652557   99836 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749649504.660074   99836 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


New https://pypi.org/project/ultralytics/8.3.153 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.129 🚀 Python-3.10.12 torch-2.5.1+cu124 CUDA:0 (NVIDIA TITAN X (Pascal), 12183MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/data22/stu236894/GitRepos/TinyML-MT/huggingface/ai_shelf/artificial_created_dataset/atrifitial_created_dataset.yaml, degrees=0.0, deterministic=True, device=2, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=400, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, mom

train: Scanning /data22/stu235269/TinyML-MT/huggingface/mvtec_yolo/labels/train.cache... 11380 images, 0 backgrounds, 0 corrupt: 100%|██████████| 11380/11380 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1294.5±453.1 MB/s, size: 142.2 KB)


val: Scanning /data22/stu235269/TinyML-MT/huggingface/mvtec_yolo/labels/val.cache... 6600 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6600/6600 [00:00<?, ?it/s]


...

      30/30      1.21G     0.4809     0.3463     0.8936         35        416: 100%|██████████| 63/63 [00:08<00:00,  7.39it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:01<00:00,  6.43it/s]

                   all        300       1410      0.992      0.994      0.993      0.853



30 epochs completed in 0.092 hours.
Optimizer stripped from runs/train/first_atrificial_dataset_full_run/weights/last.pt, 5.4MB
Optimizer stripped from runs/train/first_atrificial_dataset_full_run/weights/best.pt, 5.4MB

Validating runs/train/first_atrificial_dataset_full_run/weights/best.pt...
Ultralytics 8.3.129 🚀 Python-3.10.12 torch-2.5.1+cu124 CUDA:2 (NVIDIA TITAN X (Pascal), 12183MiB)
YOLO11n summary (fused): 100 layers, 2,583,517 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:03<00:00,  3.08it/s]


                   all        300       1410      0.992      0.994      0.993      0.854
                 apple        152        217      0.998          1      0.995      0.919
               avocado        160        228          1      0.997      0.995      0.855
                banana        122        155          1      0.998      0.995      0.669
              cucumber         98        114      0.953      0.965      0.984      0.785
             fruit tea        126        174      0.992          1      0.994      0.923
                 lemon        185        309      0.999          1      0.995      0.885
          tomato sauce        152        213      0.999          1      0.995      0.939
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/train/first_atrificial_dataset_full_run


## Create Grid for mvtec

In [7]:
import wandb
from ultralytics import YOLO
import find_usefull_images_scripts as im_script
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Modell laden (ggf. Pfad anpassen)
#model = YOLO("runs/train/YOLO11n-COCO11-first_artificial_created_dataset/weights/best.pt")

# Alle Beispielbilder laden
image_paths, _ = im_script.get_mvtec_images_for_first_artificial_dataset_classes()

batch_size = 20
num_grids = 10

for grid_idx in range(num_grids):
    start_idx = grid_idx * batch_size
    end_idx = start_idx + batch_size
    selected_paths = image_paths[start_idx:end_idx]

    # Vorhersagen durchführen (Batch)
    preds = model.predict(
        selected_paths,
        imgsz=400,
        save=False,
        stream=False
    )

    # Bilder vorbereiten
    images_drawn = []
    for img_path, pred in zip(selected_paths, preds):
        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        h, w, _ = img.shape
        for box, cls, conf in zip(pred.boxes.xyxy, pred.boxes.cls, pred.boxes.conf):
            x1, y1, x2, y2 = map(int, box)
            class_name = model.names[int(cls)]
            label = f"{class_name} {conf:.2f}"

            # Rechteck zeichnen
            cv2.rectangle(img, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2)

            # Textgröße bestimmen
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 1.2
            thickness = 2
            (text_w, text_h), baseline = cv2.getTextSize(label, font, font_scale, thickness)

            # Textposition
            text_x = x1
            if y1 - text_h - baseline > 0:
                text_y = y1 - 5
                # Hintergrundrechteck für Text (oben)
                cv2.rectangle(img, (text_x, text_y - text_h - baseline), (text_x + text_w, text_y + baseline), (0, 255, 0), cv2.FILLED)
                cv2.putText(img, label, (text_x, text_y), font, font_scale, (0, 0, 0), thickness)
            else:
                text_y = y2 + text_h + 5
                if text_y > h:
                    text_y = y2 - 5
                # Hintergrundrechteck für Text (unten)
                cv2.rectangle(img, (text_x, text_y - text_h - baseline), (text_x + text_w, text_y + baseline), (0, 255, 0), cv2.FILLED)
                cv2.putText(img, label, (text_x, text_y), font, font_scale, (0, 0, 0), thickness)

        images_drawn.append(img)

    # 5x4 Grid erstellen
    rows, cols = 5, 4
    fig, axs = plt.subplots(rows, cols, figsize=(12, 15), dpi=300)
    plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05, hspace=0.05, wspace=0.05)

    for i, ax in enumerate(axs.flat):
        if i < len(images_drawn):
            ax.imshow(images_drawn[i])
            ax.axis('off')
        else:
            ax.axis('off')

    # Grid als Bild speichern
    grid_img_path = f"prediction_grid_{grid_idx+1}.jpg"
    fig.savefig(grid_img_path, bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    # Bild bei wandb loggen
    wandb.log({f"prediction_grid_mvtec_dataset_{grid_idx+1}": wandb.Image(grid_img_path)})



WARNING ⚠️ imgsz=[400] must be multiple of max stride 32, updating to [416]
0: 320x416 2 avocados, 2.6ms
1: 320x416 2 tomato sauces, 2.6ms
2: 320x416 1 fruit tea, 1 tomato sauce, 2.6ms
3: 320x416 1 fruit tea, 2.6ms
4: 320x416 1 fruit tea, 1 tomato sauce, 2.6ms
5: 320x416 3 fruit teas, 2.6ms
6: 320x416 4 fruit teas, 2.6ms
7: 320x416 4 fruit teas, 1 lemon, 2.6ms
8: 320x416 1 fruit tea, 3 lemons, 2.6ms
9: 320x416 1 fruit tea, 2.6ms
10: 320x416 2 fruit teas, 2.6ms
11: 320x416 1 fruit tea, 3 tomato sauces, 2.6ms
12: 320x416 1 fruit tea, 2.6ms
13: 320x416 2 fruit teas, 2.6ms
14: 320x416 2 fruit teas, 1 tomato sauce, 2.6ms
15: 320x416 1 fruit tea, 2 lemons, 2.6ms
16: 320x416 2 fruit teas, 2.6ms
17: 320x416 1 cucumber, 1 fruit tea, 2.6ms
18: 320x416 1 fruit tea, 2.6ms
19: 320x416 2 fruit teas, 1 lemon, 2.6ms
Speed: 1.3ms preprocess, 2.6ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 416)

WARNING ⚠️ imgsz=[400] must be multiple of max stride 32, updating to [416]
0: 320x416 2 b

## calculate Metrixs on MVTEC 

In [8]:
# Metrik Initialisierenfrom ultralytics import YOLO
import find_usefull_images_scripts as im_script
from torchmetrics.detection.mean_ap import MeanAveragePrecision
import torch

# Modell laden
#model = YOLO("runs/train/YOLO11n-COCO11-first_artificial_created_dataset/weights/best.pt")

# Bilder + Labels laden
image_paths, label_lines = im_script.get_mvtec_images_for_first_artificial_dataset_classes()

# Hilfsfunktion: parse label_lines zu GT-Format (list von dicts)
# Beispiel: [{"boxes": Tensor[N,4], "labels": Tensor[N]}]
def parse_labels(label_lines, image_paths):
    gt_list = []
    for lines_per_image, img_path in zip(label_lines, image_paths):
        img = cv2.imread(img_path)
        h, w = img.shape[:2]
        
        boxes = []
        labels = []
        for line in lines_per_image:
            parts = line.strip().split()
            if len(parts) >= 5:
                cls = int(parts[0])
                x_c, y_c, bw, bh = map(float, parts[1:5])
                xmin = (x_c - bw/2) * w
                ymin = (y_c - bh/2) * h
                xmax = (x_c + bw/2) * w
                ymax = (y_c + bh/2) * h
                boxes.append([xmin, ymin, xmax, ymax])
                labels.append(cls)
        if boxes:
            gt_list.append({
                "boxes": torch.tensor(boxes, dtype=torch.float32),
                "labels": torch.tensor(labels, dtype=torch.int64)
            })
        else:
            gt_list.append({
                "boxes": torch.zeros((0,4), dtype=torch.float32),
                "labels": torch.zeros((0,), dtype=torch.int64)
            })
    return gt_list

gt = parse_labels(label_lines, image_paths)

# Metrik Initialisieren
map_metric = MeanAveragePrecision(iou_type='bbox')

batch_size = 10
for i in range(0, len(image_paths), batch_size):
    batch_paths = image_paths[i:i+batch_size]
    batch_gt = gt[i:i+batch_size]

    preds = model.predict(batch_paths, imgsz=400, stream=False)

    # Formatieren der Predictions für torchmetrics
    preds_for_metric = []
    for pred in preds:
        boxes = pred.boxes.xyxy.cpu()
        scores = pred.boxes.conf.cpu()
        labels = pred.boxes.cls.cpu().to(torch.int64)
        preds_for_metric.append({
            "boxes": boxes,
            "scores": scores,
            "labels": labels
        })
    # Update Metriken
    map_metric.update(preds_for_metric, batch_gt)

# Ergebnisse berechnen
metrics = map_metric.compute()

import wandb
wandb.log({
    "test_on_mv_tec/mAP50-95": metrics["map"].item(),
    "test_on_mv_tec/mAP50": metrics["map_50"].item(),
    "test_on_mv_tec/mAP75": metrics["map_75"].item()
})

print(metrics)




WARNING ⚠️ imgsz=[400] must be multiple of max stride 32, updating to [416]
0: 320x416 2 avocados, 5.3ms
1: 320x416 2 tomato sauces, 5.3ms
2: 320x416 1 fruit tea, 1 tomato sauce, 5.3ms
3: 320x416 1 fruit tea, 5.3ms
4: 320x416 1 fruit tea, 1 tomato sauce, 5.3ms
5: 320x416 3 fruit teas, 5.3ms
6: 320x416 4 fruit teas, 5.3ms
7: 320x416 4 fruit teas, 1 lemon, 5.3ms
8: 320x416 1 fruit tea, 3 lemons, 5.3ms
9: 320x416 1 fruit tea, 5.3ms
Speed: 3.0ms preprocess, 5.3ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 416)

WARNING ⚠️ imgsz=[400] must be multiple of max stride 32, updating to [416]
0: 320x416 2 fruit teas, 1.1ms
1: 320x416 1 fruit tea, 3 tomato sauces, 1.1ms
2: 320x416 1 fruit tea, 1.1ms
3: 320x416 2 fruit teas, 1.1ms
4: 320x416 2 fruit teas, 1 tomato sauce, 1.1ms
5: 320x416 1 fruit tea, 2 lemons, 1.1ms
6: 320x416 2 fruit teas, 1.1ms
7: 320x416 1 cucumber, 1 fruit tea, 1.1ms
8: 320x416 1 fruit tea, 1.1ms
9: 320x416 2 fruit teas, 1 lemon, 1.1ms
Speed: 1.1ms preprocess,

## calculate grid for custom

In [9]:
import wandb
from ultralytics import YOLO
import find_usefull_images_scripts as im_script
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Alle Beispielbilder laden
image_paths, _ = im_script.get_custom_small_class_dataset()

batch_size = 20
num_grids = 10

for grid_idx in range(num_grids):
    start_idx = grid_idx * batch_size
    end_idx = start_idx + batch_size
    selected_paths = image_paths[start_idx:end_idx]

    # Vorhersagen durchführen (Batch)
    preds = model.predict(
        selected_paths,
        imgsz=400,
        save=False,
        stream=False
    )

    # Bilder vorbereiten
    images_drawn = []
    for img_path, pred in zip(selected_paths, preds):
        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        h, w, _ = img.shape
        for box, cls, conf in zip(pred.boxes.xyxy, pred.boxes.cls, pred.boxes.conf):
            x1, y1, x2, y2 = map(int, box)
            class_name = model.names[int(cls)]
            label = f"{class_name} {conf:.2f}"

            # Rechteck zeichnen
            cv2.rectangle(img, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2)

            # Textgröße bestimmen
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 1.2
            thickness = 2
            (text_w, text_h), baseline = cv2.getTextSize(label, font, font_scale, thickness)

            # Textposition
            text_x = x1
            if y1 - text_h - baseline > 0:
                text_y = y1 - 5
                # Hintergrundrechteck für Text (oben)
                cv2.rectangle(img, (text_x, text_y - text_h - baseline), (text_x + text_w, text_y + baseline), (0, 255, 0), cv2.FILLED)
                cv2.putText(img, label, (text_x, text_y), font, font_scale, (0, 0, 0), thickness)
            else:
                text_y = y2 + text_h + 5
                if text_y > h:
                    text_y = y2 - 5
                # Hintergrundrechteck für Text (unten)
                cv2.rectangle(img, (text_x, text_y - text_h - baseline), (text_x + text_w, text_y + baseline), (0, 255, 0), cv2.FILLED)
                cv2.putText(img, label, (text_x, text_y), font, font_scale, (0, 0, 0), thickness)

        images_drawn.append(img)

    # 5x4 Grid erstellen
    rows, cols = 5, 4
    fig, axs = plt.subplots(rows, cols, figsize=(12, 15), dpi=300)
    plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05, hspace=0.05, wspace=0.05)

    for i, ax in enumerate(axs.flat):
        if i < len(images_drawn):
            ax.imshow(images_drawn[i])
            ax.axis('off')
        else:
            ax.axis('off')

    # Grid als Bild speichern
    grid_img_path = f"prediction_grid_{grid_idx+1}.jpg"
    fig.savefig(grid_img_path, bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    # Bild bei wandb loggen
    wandb.log({f"prediction_grid_custom_dataset_batch_{grid_idx+1}": wandb.Image(grid_img_path)})



WARNING ⚠️ imgsz=[400] must be multiple of max stride 32, updating to [416]
0: 320x416 1 avocado, 1 lemon, 0.9ms
1: 320x416 1 apple, 0.9ms
2: 320x416 1 avocado, 1 lemon, 1 tomato sauce, 0.9ms
3: 320x416 2 apples, 1 avocado, 0.9ms
4: 320x416 1 apple, 0.9ms
5: 320x416 1 avocado, 0.9ms
6: 320x416 1 apple, 0.9ms
7: 320x416 1 apple, 0.9ms
8: 320x416 (no detections), 0.9ms
9: 320x416 2 apples, 1 fruit tea, 0.9ms
10: 320x416 1 apple, 0.9ms
11: 320x416 1 fruit tea, 0.9ms
12: 320x416 1 apple, 0.9ms
13: 320x416 2 apples, 1 fruit tea, 0.9ms
14: 320x416 1 fruit tea, 0.9ms
15: 320x416 1 tomato sauce, 0.9ms
16: 320x416 1 fruit tea, 0.9ms
17: 320x416 1 banana, 0.9ms
18: 320x416 (no detections), 0.9ms
19: 320x416 (no detections), 0.9ms
Speed: 1.2ms preprocess, 0.9ms inference, 0.9ms postprocess per image at shape (1, 3, 320, 416)

WARNING ⚠️ imgsz=[400] must be multiple of max stride 32, updating to [416]
0: 320x416 1 apple, 0.9ms
1: 320x416 1 apple, 1 cucumber, 1 tomato sauce, 0.9ms
2: 320x416 1 app

## calculate Metrics for Custom

In [10]:
import torch
import wandb
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

# Original Dict: Prediction class id -> GT class id
label_translation_trained_on_small = {0: 1, 1: 3, 2: 4, 3: 48, 4: 26, 5: 2, 6: 5}
# Inverse Übersetzung: GT class id -> Prediction class id
label_translation = {v: k for k, v in label_translation_trained_on_small.items()}

def translate_and_expand_counts(label_dict, translation_dict, num_classes):
    """
    Übersetzt Klassen-IDs (keys) mit translation_dict.
    Gibt einen Count-Vektor zurück mit Länge num_classes.
    label_dict: dict class_id->count (raw oder GT)
    translation_dict: mapping von rohen Klassen zu Zielklassen
    """
    counts = torch.zeros(num_classes, dtype=torch.float32)
    for cls, count in label_dict.items():
        # Klasse übersetzen, falls möglich
        mapped_cls = translation_dict.get(cls, None)
        if mapped_cls is not None and mapped_cls < num_classes:
            counts[mapped_cls] += count
        # Optional: Klassen ignorieren, die nicht übersetzt werden können
    return counts

def compute_count_metrics(gt_counts_list, pred_counts_list):
    """
    Berechnet Precision, Recall, F1 anhand der Counts über alle Bilder.
    Dabei wird Precision und Recall als Summe von korrekten Klassen-Counts / Gesamt-Counts definiert.
    """

    # Alles zu numpy arrays
    gt_arr = torch.stack(gt_counts_list).numpy()
    pred_arr = torch.stack(pred_counts_list).numpy()

    # True Positives: Min von pred und gt (Counts korrekt vorhergesagt)
    tp = np.minimum(gt_arr, pred_arr).sum(axis=1)  # pro Bild
    # Summe der vorhergesagten Counts (Pred Positives)
    pred_sum = pred_arr.sum(axis=1)
    # Summe der GT Counts (Ground Truth Positives)
    gt_sum = gt_arr.sum(axis=1)

    # Vermeidung Division durch 0
    precision_per_sample = np.divide(tp, pred_sum, out=np.zeros_like(tp), where=pred_sum!=0)
    recall_per_sample = np.divide(tp, gt_sum, out=np.zeros_like(tp), where=gt_sum!=0)

    # Mittelwerte über alle Bilder
    precision = precision_per_sample.mean()
    recall = recall_per_sample.mean()
    if precision + recall == 0:
        f1 = 0.0
    else:
        f1 = 2 * (precision * recall) / (precision + recall)

    return precision, recall, f1

# === Main ===

image_paths, label_lines = im_script.get_custom_small_class_dataset()

# Anzahl Klassen anhand GT und Prediction Translation bestimmen
all_gt_classes = [int(k) for d in label_lines for k in d.keys()]
num_classes = max(label_translation_trained_on_small.keys()) + 1
print(num_classes)


batch_size = 20

all_gt_counts = []
all_pred_counts = []

for i in range(0, len(image_paths), batch_size):
    batch_paths = image_paths[i:i+batch_size]
    batch_labels = label_lines[i:i+batch_size]  # dicts: class_id->count

    # GT counts übersetzen und sammeln
    for label_dict in batch_labels:
        # label_dict ist z.B. {1:1, 3:1, 4:2, 26:1, 48:1}
        gt_count_vec = translate_and_expand_counts(label_dict, label_translation, num_classes)
        all_gt_counts.append(gt_count_vec)

    # Predictions holen
    preds_raw = model.predict(batch_paths, imgsz=400, stream=False)

    for pred in preds_raw:
        # pred.boxes.cls: Tensor mit Klasse pro Box
        # pred.boxes.xyxy: Boxes
        pred_classes = pred.boxes.cls.cpu().tolist()

        # Count Klassen (multi-counts)
        pred_counts_dict = {}
        for c in pred_classes:
            pred_counts_dict[c] = pred_counts_dict.get(c, 0) + 1

        # Klassen übersetzen und zu Count-Vector
        pred_count_vec = translate_and_expand_counts(pred_counts_dict, label_translation_trained_on_small, num_classes)
        all_pred_counts.append(pred_count_vec)

# Metriken berechnen
precision, recall, f1 = compute_count_metrics(all_gt_counts, all_pred_counts)

print(f"Precision (counts): {precision:.3f}, Recall (counts): {recall:.3f}, F1 (counts): {f1:.3f}")

# Loggen bei wandb
wandb.log({
    "eval_small_local/precision_counts": precision,
    "eval_small_local/recall_counts": recall,
    "eval_small_local/f1_score_counts": f1
})


7

WARNING ⚠️ imgsz=[400] must be multiple of max stride 32, updating to [416]
0: 320x416 1 avocado, 1 lemon, 0.9ms
1: 320x416 1 apple, 0.9ms
2: 320x416 1 avocado, 1 lemon, 1 tomato sauce, 0.9ms
3: 320x416 2 apples, 1 avocado, 0.9ms
4: 320x416 1 apple, 0.9ms
5: 320x416 1 avocado, 0.9ms
6: 320x416 1 apple, 0.9ms
7: 320x416 1 apple, 0.9ms
8: 320x416 (no detections), 0.9ms
9: 320x416 2 apples, 1 fruit tea, 0.9ms
10: 320x416 1 apple, 0.9ms
11: 320x416 1 fruit tea, 0.9ms
12: 320x416 1 apple, 0.9ms
13: 320x416 2 apples, 1 fruit tea, 0.9ms
14: 320x416 1 fruit tea, 0.9ms
15: 320x416 1 tomato sauce, 0.9ms
16: 320x416 1 fruit tea, 0.9ms
17: 320x416 1 banana, 0.9ms
18: 320x416 (no detections), 0.9ms
19: 320x416 (no detections), 0.9ms
Speed: 1.1ms preprocess, 0.9ms inference, 0.9ms postprocess per image at shape (1, 3, 320, 416)

WARNING ⚠️ imgsz=[400] must be multiple of max stride 32, updating to [416]
0: 320x416 1 apple, 0.9ms
1: 320x416 1 apple, 1 cucumber, 1 tomato sauce, 0.9ms
2: 320x416 1 a